In [1]:
# %%timeit
import os
sc_fermi_exec = '/Users/bjm42/source/sc-fermi/upload-sourceforge/sc-fermi'
print(os.popen(sc_fermi_exec).read())

**************************************************************

   SSSS    CCCC      FFFFFF  EEEEEE   RRRR   MM     MM  IIIII
  SS   S  CC   C     FF      EE      RR   R  MMM   MMM    I
  SS      CC         FF      EE      RR  R   M MM MM M    I
   SSSS   CC     --- FFFFFF  EEEEEE  RRRR    M  MMM  M    I
      SS  CC         FF      EE      R   R   M   M   M    I
  S   SS  CC   C     FF      EE      R   RR  M   M   M    I
   SSSS    CCCC      FF      EEEEEE  R   RR  M   M   M  IIIII

Energies in eV, temperature in Kelvin, DOS in states/unit cell

------
j.buckeridge@ucl.ac.uk 2018
**************************************************************

unitcell.dat found...
(Should be cell for which DOS was determined!)
 
Volume of cell:  ****************** A^3
 
Found spin polarised system...
 
Number of electrons in system:     544.000000
         Energy gap of system:       5.904000 eV
                 Temperature :    1500.000000 K
     Number of defect species:     2
 
Defects found:
Name 

In [2]:
# %%timeit

from pymatgen.io.vasp import Poscar
from py_sc_fermi.inputs import read_input_data, read_dos_data, read_unitcell_data
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.dos import DOS

# volume = Poscar.from_file('POSCAR').structure.volume
inputs = read_input_data(filename='./input-fermi.dat')
volume = read_unitcell_data(filename='./unitcell.dat')
dos = read_dos_data('./totdos.dat', nelect=inputs['nelect'], egap=inputs['egap'])
ds = DefectSystem(defect_species=inputs['defect_species'],
                  dos=dos, 
                  temperature=inputs['temperature'], 
                  volume=volume)
print(ds)
ds.get_sc_fermi(verbose=False)


Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 1500.0 K
Number of defect species: 2
Volume of cell: 1056.622886347127 A^3
Reading spin-polarised DOS
Integration of DOS up to Fermi level: 542.8624039950018
Renormalised integrated DOS        : 544.0
DefectSystem
  nelect: 544 e
  egap:   5.904 eV
  volume: 1056.622886347127 A^3
  temperature: 1500.0 K

Contains defect species:

v_Li, nsites=3
  q=+0.0, e=3.17135738000001, deg=2
  q=-1.0, e=3.659671988940586, deg=1

Li_i, nsites=1
  q=+0.0, e=2.9462461100000685, deg=2
  q=+1.0, e=-2.6395306397551033, deg=1



3.0785979224348923

In [3]:
# %%timeit
ds.get_sc_fermi(verbose=False, emin=0.0)

3.0785979224348923

In [4]:
ds.get_sc_fermi_new(verbose=False, emin=0.0)

{'e_fermi': 3.0785979224348923,
 'n_iter': 99,
 'bracket': [3.078597922434892, 3.0785979224348923]}

In [5]:
ds.report()

SC Fermi level :      3.0785979224348923  (eV)

Concentrations:
n (electrons)  : 476031945159.1502 cm^-3
p (holes)      : 453017268174.3595 cm^-3
v_Li           : 3.1686955438653497e+19 cm^-3
Li_i           : 3.168695557495372e+19 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_Li       : Charge Concentration(cm^-3) Total
           :  0.0  1.255890e+11          0.00 
           : -1.0  3.168696e+19          100.00 
---------------------------------------------------------
Li_i       : Charge Concentration(cm^-3) Total
           :  0.0  2.388746e+11          0.00 
           :  1.0  3.168696e+19          100.00 


In [5]:
c_v_Li = ds.defect_species_by_name('v_Li').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [6]:
c_Li_i = ds.defect_species_by_name('Li_i').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [7]:
(c_v_Li - c_Li_i)

-9.020562075079397e-17

In [8]:
ds.get_sc_fermi()

     fun: 3.0159137062724106e-09
 message: 'Solution found.'
    nfev: 39
  status: 0
 success: True
       x: 3.3787971647280033


3.3787971647280033

In [9]:
ds.get_constrained_sc_fermi(constraint={'v_Li': +1, 'Li_i': -1}, total=-0.3)

/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))


3.096792116700261